# جنریٹو نیٹ ورکس

ریکرنٹ نیورل نیٹ ورکس (RNNs) اور ان کے گیٹڈ سیل ویریئنٹس جیسے لانگ شارٹ ٹرم میموری سیلز (LSTMs) اور گیٹڈ ریکرنٹ یونٹس (GRUs) نے زبان ماڈلنگ کے لیے ایک طریقہ فراہم کیا، یعنی یہ الفاظ کی ترتیب سیکھ سکتے ہیں اور کسی ترتیب میں اگلے لفظ کی پیش گوئی کر سکتے ہیں۔ اس سے ہمیں RNNs کو **جنریٹو کاموں** کے لیے استعمال کرنے کی اجازت ملتی ہے، جیسے عام متن کی تخلیق، مشین ترجمہ، اور یہاں تک کہ تصویر کی وضاحت۔

پچھلے یونٹ میں جس RNN آرکیٹیکچر پر ہم نے بات کی، اس میں ہر RNN یونٹ اگلی چھپی ہوئی حالت کو بطور آؤٹ پٹ پیدا کرتا تھا۔ تاہم، ہم ہر ریکرنٹ یونٹ میں ایک اور آؤٹ پٹ بھی شامل کر سکتے ہیں، جو ہمیں ایک **ترتیب** آؤٹ پٹ کرنے کی اجازت دیتا ہے (جو اصل ترتیب کے برابر لمبائی میں ہوتی ہے)۔ مزید برآں، ہم ایسے RNN یونٹس استعمال کر سکتے ہیں جو ہر قدم پر ان پٹ قبول نہیں کرتے، بلکہ صرف ایک ابتدائی حالت ویکٹر لیتے ہیں، اور پھر آؤٹ پٹس کی ایک ترتیب پیدا کرتے ہیں۔

اس نوٹ بک میں، ہم سادہ جنریٹو ماڈلز پر توجہ مرکوز کریں گے جو ہمیں متن تخلیق کرنے میں مدد دیتے ہیں۔ سادگی کے لیے، آئیے ایک **کریکٹر لیول نیٹ ورک** بنائیں، جو متن کو حرف بہ حرف تخلیق کرتا ہے۔ تربیت کے دوران، ہمیں کچھ متن کا مجموعہ لینا ہوگا اور اسے حرفوں کی ترتیب میں تقسیم کرنا ہوگا۔


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## کردار کی لغت بنانا

کردار کی سطح پر جنریٹو نیٹ ورک بنانے کے لیے، ہمیں متن کو الفاظ کے بجائے انفرادی کرداروں میں تقسیم کرنا ہوگا۔ یہ ایک مختلف ٹوکنائزر کی تعریف کرکے کیا جا سکتا ہے:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


آئیے دیکھتے ہیں کہ ہم اپنے ڈیٹاسیٹ سے متن کو کیسے انکوڈ کر سکتے ہیں:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## جنریٹو RNN کی تربیت

ہم RNN کو ٹیکسٹ جنریٹ کرنے کے لیے اس طرح تربیت دیں گے۔ ہر قدم پر، ہم `nchars` لمبائی کے کرداروں کی ایک ترتیب لیں گے، اور نیٹ ورک سے کہیں گے کہ ہر ان پٹ کردار کے لیے اگلا آؤٹ پٹ کردار پیدا کرے:

![تصویر میں RNN کے ذریعے 'HELLO' لفظ کی جنریشن کا ایک مثال دکھایا گیا ہے۔](../../../../../translated_images/ur/rnn-generate.56c54afb52f9781d.webp)

حقیقی منظرنامے کے مطابق، ہم کچھ خاص کرداروں کو بھی شامل کرنا چاہ سکتے ہیں، جیسے *end-of-sequence* `<eos>`۔ ہمارے معاملے میں، ہم صرف نیٹ ورک کو لامتناہی ٹیکسٹ جنریشن کے لیے تربیت دینا چاہتے ہیں، اس لیے ہم ہر ترتیب کا سائز `nchars` ٹوکنز کے برابر مقرر کریں گے۔ نتیجتاً، ہر تربیتی مثال `nchars` ان پٹس اور `nchars` آؤٹ پٹس پر مشتمل ہوگی (جو ان پٹ ترتیب کو ایک علامت بائیں طرف منتقل کرنے سے حاصل ہوں گے)۔ منی بیچ کئی ایسی ترتیبوں پر مشتمل ہوگا۔

ہم منی بیچز کو اس طرح جنریٹ کریں گے کہ ہر خبر کے متن کی لمبائی `l` لے کر، اس سے تمام ممکنہ ان پٹ-آؤٹ پٹ امتزاجات پیدا کریں گے (ایسے امتزاجات کی تعداد `l-nchars` ہوگی)۔ یہ ایک منی بیچ بنائیں گے، اور منی بیچز کا سائز ہر تربیتی قدم پر مختلف ہوگا۔


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

اب ہم جنریٹر نیٹ ورک کی تعریف کرتے ہیں۔ یہ کسی بھی ری کرنٹ سیل پر مبنی ہو سکتا ہے جس پر ہم نے پچھلے یونٹ میں بات کی تھی (سادہ، LSTM یا GRU)۔ ہماری مثال میں ہم LSTM استعمال کریں گے۔

چونکہ نیٹ ورک حروف کو ان پٹ کے طور پر لیتا ہے، اور الفاظ کا سائز کافی چھوٹا ہے، ہمیں ایمبیڈنگ لیئر کی ضرورت نہیں ہے، ون-ہاٹ-انکوڈڈ ان پٹ براہ راست LSTM سیل میں جا سکتا ہے۔ تاہم، چونکہ ہم حروف کے نمبر ان پٹ کے طور پر پاس کرتے ہیں، ہمیں انہیں LSTM میں پاس کرنے سے پہلے ون-ہاٹ-انکوڈ کرنا ہوگا۔ یہ `forward` پاس کے دوران `one_hot` فنکشن کو کال کر کے کیا جاتا ہے۔ آؤٹ پٹ انکوڈر ایک لکیری لیئر ہوگی جو چھپے ہوئے اسٹیٹ کو ون-ہاٹ-انکوڈڈ آؤٹ پٹ میں تبدیل کرے گی۔


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

تربیت کے دوران، ہم چاہتے ہیں کہ ہم پیدا شدہ متن کو نمونہ کے طور پر لے سکیں۔ اس کے لیے، ہم `generate` فنکشن کو ڈیفائن کریں گے جو کہ ایک آؤٹ پٹ سٹرنگ پیدا کرے گا جس کی لمبائی `size` ہوگی، اور یہ ابتدائی سٹرنگ `start` سے شروع ہوگی۔

اس کا طریقہ کار کچھ یوں ہے۔ سب سے پہلے، ہم پوری `start` سٹرنگ کو نیٹ ورک کے ذریعے پاس کریں گے، اور آؤٹ پٹ اسٹیٹ `s` اور اگلا پیش گوئی کردہ کردار `out` لیں گے۔ چونکہ `out` ایک ون-ہاٹ انکوڈڈ ہے، ہم `argmax` استعمال کریں گے تاکہ کردار `nc` کا انڈیکس لغت میں حاصل کیا جا سکے، اور `itos` استعمال کرتے ہوئے اصل کردار معلوم کریں گے اور اسے کرداروں کی نتیجہ خیز فہرست `chars` میں شامل کریں گے۔ ایک کردار پیدا کرنے کا یہ عمل `size` بار دہرایا جائے گا تاکہ مطلوبہ تعداد میں کردار پیدا کیے جا سکیں۔


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

چلو تربیت شروع کرتے ہیں! تربیتی لوپ تقریباً ویسا ہی ہے جیسا کہ ہمارے پچھلے تمام مثالوں میں تھا، لیکن اس بار درستگی کے بجائے ہم ہر 1000 ایپوکز کے بعد نمونہ شدہ تیار کردہ متن پرنٹ کریں گے۔

خاص توجہ اس بات پر دینی ہوگی کہ ہم نقصان (loss) کو کیسے حساب کرتے ہیں۔ ہمیں نقصان کو ایک-ہاٹ-انکوڈڈ آؤٹ پٹ `out` اور متوقع متن `text_out`، جو کہ کریکٹر انڈیکسز کی فہرست ہے، کے ساتھ حساب کرنا ہوگا۔ خوش قسمتی سے، `cross_entropy` فنکشن غیر معمولی نیٹ ورک آؤٹ پٹ کو پہلے آرگومنٹ کے طور پر اور کلاس نمبر کو دوسرے آرگومنٹ کے طور پر قبول کرتا ہے، جو بالکل وہی ہے جو ہمارے پاس ہے۔ یہ خودکار طور پر منی بیچ کے سائز پر اوسط بھی نکالتا ہے۔

ہم تربیت کو `samples_to_train` نمونوں تک محدود کرتے ہیں تاکہ زیادہ انتظار نہ کرنا پڑے۔ ہم آپ کو تجربہ کرنے اور طویل تربیت آزمانے کی ترغیب دیتے ہیں، ممکنہ طور پر کئی ایپوکز کے لیے (ایسی صورت میں آپ کو اس کوڈ کے ارد گرد ایک اور لوپ بنانا ہوگا)۔


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

یہ مثال پہلے ہی کافی اچھا متن تیار کرتی ہے، لیکن اسے کئی طریقوں سے مزید بہتر بنایا جا سکتا ہے:

* **بہتر منی بیچ جنریشن**۔ جس طریقے سے ہم نے تربیت کے لیے ڈیٹا تیار کیا، وہ ایک نمونے سے ایک منی بیچ تیار کرنا تھا۔ یہ مثالی نہیں ہے، کیونکہ منی بیچز کے سائز مختلف ہوتے ہیں، اور کچھ تو تیار ہی نہیں ہو سکتے کیونکہ متن `nchars` سے چھوٹا ہوتا ہے۔ اس کے علاوہ، چھوٹے منی بیچز GPU کو مؤثر طریقے سے استعمال نہیں کرتے۔ زیادہ بہتر ہوگا کہ تمام نمونوں سے ایک بڑا متن کا حصہ لیا جائے، پھر تمام ان پٹ-آؤٹ پٹ جوڑوں کو تیار کیا جائے، انہیں شفل کیا جائے، اور برابر سائز کے منی بیچز تیار کیے جائیں۔

* **ملٹی لیئر LSTM**۔ 2 یا 3 لیئرز کے LSTM سیلز آزمانا سمجھداری ہوگی۔ جیسا کہ ہم نے پچھلے یونٹ میں ذکر کیا تھا، LSTM کی ہر لیئر متن سے مخصوص پیٹرنز نکالتی ہے، اور کریکٹر لیول جنریٹر کے معاملے میں ہم توقع کر سکتے ہیں کہ نچلی LSTM لیئر حرفوں کو نکالنے کی ذمہ دار ہوگی، اور اعلیٰ لیئرز الفاظ اور الفاظ کے مجموعے کے لیے۔ یہ آسانی سے LSTM کنسٹرکٹر کو نمبر آف لیئرز پیرامیٹر پاس کر کے نافذ کیا جا سکتا ہے۔

* آپ **GRU یونٹس** کے ساتھ تجربہ کرنا بھی چاہ سکتے ہیں اور دیکھ سکتے ہیں کہ کون سا بہتر کام کرتا ہے، اور **مختلف چھپی ہوئی لیئرز کے سائز** کے ساتھ۔ بہت بڑی چھپی ہوئی لیئر اوورفٹنگ کا باعث بن سکتی ہے (مثلاً نیٹ ورک بالکل متن سیکھ لے گا)، اور چھوٹا سائز اچھا نتیجہ پیدا نہیں کر سکتا۔


## نرم متن کی تخلیق اور درجہ حرارت

`generate` کی پچھلی تعریف میں، ہم ہمیشہ اگلے کردار کے طور پر سب سے زیادہ امکان والے کردار کو منتخب کر رہے تھے۔ اس کا نتیجہ یہ نکلا کہ متن اکثر بار بار ایک ہی کردار کے سلسلوں کے درمیان "چکر" لگاتا رہا، جیسے اس مثال میں:
```
today of the second the company and a second the company ...
```

تاہم، اگر ہم اگلے کردار کے لیے احتمال کی تقسیم کو دیکھیں، تو یہ ممکن ہے کہ چند سب سے زیادہ احتمالات کے درمیان فرق زیادہ نہ ہو، مثلاً ایک کردار کا احتمال 0.2 ہو سکتا ہے، اور دوسرے کا 0.19، وغیرہ۔ مثال کے طور پر، جب '*play*' کے سلسلے میں اگلے کردار کی تلاش کی جائے، تو اگلا کردار برابر طور پر اسپیس یا **e** ہو سکتا ہے (جیسا کہ لفظ *player* میں)۔

یہ ہمیں اس نتیجے پر پہنچاتا ہے کہ ہمیشہ سب سے زیادہ احتمال والے کردار کو منتخب کرنا "منصفانہ" نہیں ہوتا، کیونکہ دوسرے سب سے زیادہ احتمال والے کردار کو منتخب کرنا بھی ہمیں بامعنی متن تک لے جا سکتا ہے۔ زیادہ دانشمندی یہ ہے کہ نیٹ ورک کے آؤٹ پٹ سے دی گئی احتمال کی تقسیم سے کرداروں کو **نمونہ** کے طور پر منتخب کیا جائے۔

یہ نمونہ `multinomial` فنکشن کے ذریعے کیا جا سکتا ہے، جو کہ **multinomial distribution** کو نافذ کرتا ہے۔ ایک فنکشن جو اس **نرم** متن کی تخلیق کو نافذ کرتا ہے، نیچے بیان کیا گیا ہے:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

ہم نے ایک اور پیرامیٹر متعارف کروایا ہے جسے **temperature** کہا جاتا ہے، جو یہ ظاہر کرنے کے لیے استعمال ہوتا ہے کہ ہمیں سب سے زیادہ احتمال کے ساتھ کتنی سختی سے جڑے رہنا چاہیے۔ اگر temperature 1.0 ہو، تو ہم منصفانہ multinomial sampling کرتے ہیں، اور جب temperature لامحدود ہو جاتا ہے - تمام احتمالات برابر ہو جاتے ہیں، اور ہم اگلا کردار بے ترتیب منتخب کرتے ہیں۔ نیچے دیے گئے مثال میں ہم دیکھ سکتے ہیں کہ جب ہم temperature کو بہت زیادہ بڑھاتے ہیں تو متن بے معنی ہو جاتا ہے، اور جب یہ 0 کے قریب ہو جاتا ہے تو یہ "cycled" سخت پیدا کردہ متن سے مشابہت رکھتا ہے۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
